In [ ]:
import os
import json
import shutil
from tqdm import tqdm
from dotenv import load_dotenv
from PIL import Image

In [ ]:
load_dotenv()

DATASET_DIR = os.getenv("DATASET_DIR")
ANNOTATION_PATH = os.getenv("ANNOTATION_PATH")
OUTPUT_DIR = os.getenv("OUTPUT_DIR")
TRUST_THRESHOLD = float(os.getenv("TRUST_THRESHOLD", 0.5))

os.makedirs(OUTPUT_DIR, exist_ok=True)

print("📁 Dataset directory:", DATASET_DIR)
print("📜 Annotation path:", ANNOTATION_PATH)
print("📂 Output directory:", OUTPUT_DIR)
print("🎯 Trust threshold:", TRUST_THRESHOLD)

In [ ]:
# 2️⃣ Load Annotations
print("📂 Loading annotations...")
with open(ANNOTATION_PATH, "r") as f:
    data = json.load(f)

print(f"✅ Loaded {len(data)} image entries.")

In [ ]:
# 3️⃣ Define Helper Functions
def is_valid_bbox(bbox, img_w, img_h):
    """Check if a bounding box is inside image bounds."""
    left, top, width, height = bbox["left"], bbox["top"], bbox["width"], bbox["height"]
    return (
        left >= 0
        and top >= 0
        and width > 0
        and height > 0
        and left + width <= img_w
        and top + height <= img_h
    )


def is_valid_part(part, img_w, img_h):
    """Check if part has enough trust and valid bbox."""
    trust = float(part.get("trust", 0.0))  # default 1.0 if missing
    if trust < TRUST_THRESHOLD:
        return False
    if "absolute_bounding_box" not in part:
        return False
    if not is_valid_bbox(part["absolute_bounding_box"], img_w, img_h):
        return False
    return True

In [ ]:
# 4️⃣ Filter Annotations
print("🧠 Cleaning dataset...")
cleaned_data = {}
removed_images = 0

for img_id, info in tqdm(data.items()):
    img_w = info["image"]["width"]
    img_h = info["image"]["height"]
    parts = info.get("parts", {})

    # Keep only valid parts
    valid_parts = {
        name: p for name, p in parts.items()
        if is_valid_part(p, img_w, img_h)
    }

    # Skip images with no valid parts
    if not valid_parts:
        removed_images += 1
        continue

    # Save cleaned entry
    info["parts"] = valid_parts
    cleaned_data[img_id] = info

print(f"✅ Finished cleaning. {removed_images} images removed.")
print(f"📸 Remaining images: {len(cleaned_data)}")

In [ ]:
# 5️⃣ Copy Valid Images
print("📁 Copying valid images...")
for img_id in tqdm(cleaned_data.keys()):
    src_img_path = os.path.join(DATASET_DIR, img_id)
    dst_img_path = os.path.join(OUTPUT_DIR, img_id)

    if os.path.exists(src_img_path):
        shutil.copy(src_img_path, dst_img_path)

In [ ]:
# 6️⃣ Save Cleaned JSON

CLEAN_JSON_PATH = os.path.join(OUTPUT_DIR, "annotations_cleaned.json")

with open(CLEAN_JSON_PATH, "w") as f:
    json.dump(cleaned_data, f, indent=2)

print("✅ Cleaned JSON saved at:", CLEAN_JSON_PATH)
print("✅ Cleaned images saved in:", OUTPUT_DIR)